In [1]:
!pip install spotipy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 6.6 MB/s eta 0:00:00


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import numpy as np

import plotly.express as px

In [3]:
client_id= 'ebceac8bffab4afcaf5ec63bb0b23500'

client_secret= '6a0a0bbeb2064146ba73764ef91f6889'

client_credentials_manager= SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

sp= spotipy.Spotify(client_credentials_manager= client_credentials_manager)

In [19]:
#funcion para buscar el id del artista con el nombre de este pasado por parametro
artist = 'The Weeknd'
def obtenerId(artista):
    resultado = sp.search(q=artista, limit= 1, type= 'artist') #limitamos a uno la busqueda del artista
    #navegando entre diccionarios para encontrar el id del artista
    return resultado['artists']['items'][0]['id']

print(obtenerId(artist))


1Xyo4u8uXC1ZmMpatF05PJ


In [20]:
#practicando con la librería
artista_id= obtenerId(artist)
albums = sp.artist_albums(artista_id, album_type='album')

for album in albums['items']:
  print(album['name'])

Starboy (Deluxe)
Live At SoFi Stadium
Avatar: The Way of Water (Original Motion Picture Soundtrack)
Dawn FM (Alternate World)
Dawn FM
After Hours (Deluxe)
After Hours
My Dear Melancholy,
Starboy
Beauty Behind The Madness
Kiss Land (Deluxe)
Kiss Land
Trilogy
Echoes Of Silence (Original)
Thursday (Original)
House Of Balloons (Original)


In [26]:
artist = 'The Weeknd'
def obtenerId(artista):
    resultado = sp.search(q=artista, limit= 1, type= 'artist') #limitamos a uno la busqueda del artista
    #navegando entre diccionarios para encontrar el id del artista
    return resultado['artists']['items'][0]['id']

artista_id= obtenerId(artist)
def obtenerData(artista_id):
    canciones_data = []

    albumes= sp.artist_albums(artista_id, album_type='album') #obtener los albumes del artista

    for album in albumes['items']:
      album_nombre= album['name']
      album_tipo= {'album':'Álbum'}[album['album_type']]
      album_año= album['release_date'].split('-')[0] #obteniendo solo año de lanzamiento

      tracks = sp.album_tracks(album['id'])

      for track in tracks['items']: #Por cada cancion, recorrer el diccionario de items
        cancion_nombre = track['name']
        cancion_artistas = ', '.join([t['name'] for t in track['artists']])
        cancion_duracion = '{:0d}:{:0d}'.format(*divmod(track['duration_ms']//1000, 60)) #convertir los milisegundos en minutos y segundos
        cancion_popularidad = sp.track(track['id'])['popularity'] #obtener la popularidad de la cancion

        #añadir los detalles de cada cancion
        canciones_data.append([album_nombre, album_tipo, album_año, cancion_nombre, cancion_artistas, cancion_duracion, cancion_popularidad])

    return pd.DataFrame(canciones_data, columns=['Álbum', 'Tipo', 'Año', 'Canción', 'Artistas', 'Duración', 'Popularidad'])



In [29]:
#asignando a df el data frame que retorna la funcion
df= obtenerData(artista_id)
#realizando una copia del dataFrame dentro de df_canciones para su posterior uso
df_canciones=df.copy()
#convirtiendo la duracion de
df_canciones['Duracion_seg']= df_canciones['Duración'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))

df_canciones.head()

,Álbum,Tipo,Año,Canción,Artistas,Duración,Popularidad,Duracion_seg
0,Starboy (Deluxe),Álbum,2023,Starboy (feat. Daft Punk),"The Weeknd, Daft Punk",3:50,63,230
1,Starboy (Deluxe),Álbum,2023,Party Monster,The Weeknd,4:9,61,249
2,Starboy (Deluxe),Álbum,2023,False Alarm,The Weeknd,3:40,59,220
3,Starboy (Deluxe),Álbum,2023,Reminder,The Weeknd,3:38,60,218
4,Starboy (Deluxe),Álbum,2023,Rockin’,The Weeknd,3:52,57,232


In [40]:
#canciones más populares

canciones_populares = df_canciones.sort_values(by=['Popularidad'], ascending=False).head(10)

color = {'Starboy': '#C0392B', 'After Hours':'#2E4053', 'Beauty Behind The Madness': '#5F6A6A', 'My Dear Melancholy,':'#D35400', 'Dawn FM':'#212F3D'}

#creando grafico de barras
fig = px.bar(canciones_populares,
             x= 'Canción',
             y='Popularidad',
             text= 'Popularidad',
             color='Álbum',
             color_discrete_map=color,
             hover_data={'Álbum':True, 'Año':True},
             title='Canciones más populares',
             template= 'plotly_dark')

#ordenar la categoría del eje x de acuerdo a la popularidad

fig.update_layout(title={'text':'<i>Canciones más populares</i>',
                         'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor':'top',
                        'font': dict(size=20, color='white')},
                  xaxis={'categoryorder':'total descending'})

#establecemos el rango de y
fig.update_yaxes(range=[75, canciones_populares['Popularidad'].max()+1], showgrid=False, showticklabels=False)

#ajustar posicion y formato
fig.update_traces(textposition='outside')

fig.show()
